In [ ]:
import sys
import sounddevice as sd
import soundcard as sc
import soundfile as sf
import numpy as np
import wave
import warnings
import subprocess
import os
import sounddevice as sd
import soundfile as sf

from PyQt5.QtWidgets import QApplication, QVBoxLayout, QWidget, QPushButton, QMainWindow, QLabel
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtCore import QUrl, Qt, QThread, QTimer
from PyQt5.QtGui import QPixmap
from pytube import YouTube
from datetime import datetime



# Suppress specific warnings related to data discontinuity
warnings.filterwarnings("ignore", category=sc.SoundcardRuntimeWarning)

os.makedirs("Recording", exist_ok=True)
os.makedirs("RecordingHistory", exist_ok=True)

# Global constants
SAMPLE_RATE = 48000  # [Hz], sampling rate.
chunk_size = 4096    # Increased size to reduce handling frequency

# Thread for microphone recording
class MicrophoneRecorder(QThread):
    def __init__(self):
        super().__init__()
        self.fs = 44100  # Sample rate
        self.channels = 1  # Mono (Microphone)
        self.recording = False
        self.frames = []

    def run(self):
        self.frames = []
        self.recording = True
        with sd.InputStream(channels=self.channels, samplerate=self.fs, callback=self.callback):
            while self.recording:
                sd.sleep(100)

    def callback(self, indata, frames, time, status):
        if status:
            print(status)
        self.frames.append(indata.copy())

    def stop_recording(self):
        self.recording = False
        self.quit()
        self.wait()

    def save_recording(self, filename="microphone_output.wav"):
            audio_data = np.concatenate(self.frames, axis=0)
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            # Paths for saving recordings
            recent_recording_path = "Recording/microphone_output_1.wav"
            history_filename = f"RecordingHistory/microphone_output_{timestamp}.wav"

            # Save the recent recording (overwrite)
            with wave.open(recent_recording_path, 'wb') as wf:
                wf.setnchannels(self.channels)
                wf.setsampwidth(2)
                wf.setframerate(self.fs)
                wf.writeframes(audio_data.tobytes())

            # Save a copy in the 'RecordingHistory' folder with timestamp
            with wave.open(history_filename, 'wb') as wf:
                wf.setnchannels(self.channels)
                wf.setsampwidth(2)
                wf.setframerate(self.fs)
                wf.writeframes(audio_data.tobytes())

            print(f"Microphone recording saved to {recent_recording_path} and {history_filename}")

# Thread for system sound recording using soundcard library
class SystemAudioRecorder(QThread):
    def __init__(self):
        super().__init__()
        self.mic_frames = []
        self.sys_frames = []
        self.recording = False

    def run(self):
        self.mic_frames = []
        self.sys_frames = []
        self.recording = True
        microphone = sc.default_microphone()
        speaker = sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True)

        with microphone.recorder(samplerate=SAMPLE_RATE, blocksize=chunk_size) as mic_recorder, \
             speaker.recorder(samplerate=SAMPLE_RATE, blocksize=chunk_size) as sys_recorder:

            while self.recording:
                try:
                    mic_data = mic_recorder.record(numframes=chunk_size)
                    sys_data = sys_recorder.record(numframes=chunk_size)

                    self.mic_frames.append(mic_data)
                    self.sys_frames.append(sys_data)

                except sc.SoundcardRuntimeWarning as e:
                    print(f"Warning caught: {e}")
                    continue

    def stop_recording(self):
        self.recording = False
        self.quit()
        self.wait()

    def save_recording(self, mic_filename="microphone_output.wav", sys_filename="system_output.wav"):
            mic_recorded_data = np.concatenate(self.mic_frames, axis=0)
            sys_recorded_data = np.concatenate(self.sys_frames, axis=0)
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            # Paths for saving recordings
            recent_mic_path = "Recording/microphone_output_1.wav"
            recent_sys_path = "Recording/system_output_1.wav"
            history_mic_filename = f"RecordingHistory/microphone_output_{timestamp}.wav"
            history_sys_filename = f"RecordingHistory/system_output_{timestamp}.wav"

            # Save the recent recordings (overwrite)
            sf.write(file=recent_mic_path, data=mic_recorded_data[:, 0], samplerate=SAMPLE_RATE)
            sf.write(file=recent_sys_path, data=sys_recorded_data[:, 0], samplerate=SAMPLE_RATE)

            # Save copies in the 'RecordingHistory' folder with timestamp
            sf.write(file=history_mic_filename, data=mic_recorded_data[:, 0], samplerate=SAMPLE_RATE)
            sf.write(file=history_sys_filename, data=sys_recorded_data[:, 0], samplerate=SAMPLE_RATE)

            print(f"Microphone recording saved to {recent_mic_path} and {history_mic_filename}")
            print(f"System audio recording saved to {recent_sys_path} and {history_sys_filename}")

#Calling Scoring ALgorithm
class MatlabRunner(QThread):
    def __init__(self, script_name):
        super().__init__()
        self.script_name = script_name

    def run(self):
        try:
            # Define the MATLAB executable path (adjust based on your installation path)
            matlab_path = r"D:\Softwares\bin\matlab.exe"

             # Command to execute the MATLAB script in batch mode
            command = [matlab_path, "-batch", self.script_name]

            # Run the command in the Windows Terminal
            result = subprocess.run(command, capture_output=True, text=True)

            # Print the output from MATLAB
            if result.returncode == 0:
                print(f"MATLAB script '{self.script_name}' executed successfully.")
            else:
                print(f"MATLAB script '{self.script_name}' failed with exit code {result.returncode}.")
        except Exception as e:
            print(f"Error running MATLAB script: {e}")

# Main Window for the PyQt application
class MainWindow(QMainWindow):
    def __init__(self):

        self.save_recordings_flag = False  # Flag to determine whether to save recordings
        super().__init__()

        self.setWindowTitle("KARAOKEPY")
        self.setGeometry(300, 200, 800, 800)


        self.layout = QVBoxLayout()

        self.setStyleSheet("""
        background-color: black;
        background-image: url(karaoke-bg.jpg);
        background-repeat: no-repeat;
        background-position: center;
    """)
        

        # Add image
        self.image_label = QLabel(self)
        pixmap = QPixmap("banner-new.gif")
        self.image_label.setPixmap(pixmap)
        self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        self.image_label.setAlignment(Qt.AlignCenter)

        # Add browser view
        self.browser = QWebEngineView()
        self.browser.setUrl(QUrl("https://www.youtube.com/watch?v=maTP315XZCQ"))

        # Add Start/Stop button
        self.record_button = QPushButton("Start Karaoke", self)
        self.record_button.setFixedHeight(80)
        self.record_button.setStyleSheet("background-color: green; color: white")
        self.record_button.clicked.connect(self.toggle_karaoke)

        # Add widgets to layout
        self.layout.addWidget(self.image_label, 15)
        self.layout.addWidget(self.browser, 70)
        self.layout.addWidget(self.record_button, 15)

        central_widget = QWidget()
        central_widget.setLayout(self.layout)
        self.setCentralWidget(central_widget)

        # Initialize state variables
        self.karaoke_running = False
        self.microphone_recorder = MicrophoneRecorder()
        self.system_audio_recorder = SystemAudioRecorder()
        self.recording_counter = 1
        self.remaining_seconds = 0

    def toggle_karaoke(self):
        if not self.karaoke_running:
            # Stop any existing timer and reset remaining seconds
            if hasattr(self, 'remaining_timer') and self.remaining_timer.isActive():
                self.remaining_timer.stop()

            self.remaining_seconds = 0  # Reset remaining seconds

            # Start karaoke mode
            self.browser.setEnabled(False)
            self.record_button.setText("Stop Karaoke")
            self.record_button.setStyleSheet("background-color: red; color: white")
            self.karaoke_running = True
            self.save_recordings_flag = True  # Allow saving when karaoke starts

            # Start both microphone and system audio recording
            self.microphone_recorder.start()
            self.system_audio_recorder.start()

            # Fetch the initial video duration and start the timer
            self.fetch_video_duration()
        else:
            # Stop karaoke mode
            self.browser.setEnabled(True)
            self.record_button.setText("Start Karaoke")
            self.record_button.setStyleSheet("background-color: green; color: white")
            self.karaoke_running = False

            # Stop both recordings
            self.microphone_recorder.stop_recording()
            self.system_audio_recorder.stop_recording()

            # Stop the timer and reset the remaining seconds
            if hasattr(self, 'remaining_timer'):
                self.remaining_timer.stop()
            self.remaining_seconds = 0
            self.save_recordings_flag = False  # Prevent saving when stopped manually

            print("Karaoke stopped, timer reset.")

    def fetch_video_duration(self):
        # JavaScript to get the current and total duration from the YouTube video player
        js_code = """
        var currentTime = document.getElementsByClassName('ytp-time-current')[0].textContent;
        var totalTime = document.getElementsByClassName('ytp-time-duration')[0].textContent;
        currentTime + '|' + totalTime;
        """
        self.browser.page().runJavaScript(js_code, self.initialize_remaining_time)

    def initialize_remaining_time(self, time_string):
        # Reset remaining seconds before calculating the new duration
        self.remaining_seconds = 0

        # Calculate total video duration from fetched JavaScript time
        current_time, total_time = time_string.split('|')

        def time_to_seconds(time):
            minutes, seconds = map(int, time.split(':'))
            return minutes * 60 + seconds

        current_seconds = time_to_seconds(current_time)
        total_seconds = time_to_seconds(total_time)
        self.remaining_seconds = total_seconds - current_seconds

        # Start the 1-second interval timer
        self.start_remaining_time_timer()

    def start_remaining_time_timer(self):
        # Check remaining time every second
        self.remaining_timer = QTimer()
        self.remaining_timer.timeout.connect(self.update_remaining_time)
        self.remaining_timer.start(1000)


    #FINSIHED LOGIC:
    def update_remaining_time(self):
        if self.remaining_seconds > 0:
            self.remaining_seconds -= 1
            print(f"Remaining time: {self.remaining_seconds} seconds")

            # Save recordings and call MATLAB scoring when the remaining time is exactly 10 seconds
            if self.remaining_seconds == 10 and self.save_recordings_flag:
                print("Saving recordings and calling MATLAB scoring script...")
                self.microphone_recorder.stop_recording()
                self.system_audio_recorder.stop_recording()

                mic_filename = f"microphone_output_{self.recording_counter}.wav"
                sys_filename = f"system_output_{self.recording_counter}.wav"

                self.microphone_recorder.save_recording(mic_filename)
                self.system_audio_recorder.save_recording(mic_filename, sys_filename)

                # Run the MATLAB script asynchronously using a separate thread
                self.matlab_thread = MatlabRunner("ScoringAlgorithm")
                self.matlab_thread.start()

                # Prevent further saves
                self.save_recordings_flag = False
        else:
            print("Song finished!")
            self.remaining_timer.stop()
            self.toggle_karaoke()  # Stop karaoke mode automatically when time is up
            #$print("Showing UI for score (to be implemented).")
            self.show_score_ui()


    def show_score_ui(self):
        try:
            # Read the score from KaraokeScore.txt
            score_file = "Recording/KaraokeScore.txt"
            with open(score_file, "r") as file:
                score = file.read().strip()

            # Hide the YouTube view, Start/Stop button, and banner image
            self.browser.hide()
            self.record_button.hide()
            self.image_label.hide()

            # Create a score label
            self.score_label = QLabel(self)
            self.score_label.setText(f"🎉 Your Score: {score} 🎉")
            self.score_label.setAlignment(Qt.AlignCenter)
            self.score_label.setStyleSheet("color: gold; font-weight: bold;")

            # Create vertical spacers for centering
            # self.layout.addStretch()
            self.layout.addWidget(self.score_label)
            # self.layout.addStretch()

            # Play celebratory sound
            self.play_celebratory_sound()

            # Restore the UI after 10 seconds
            QTimer.singleShot(10000, self.restore_ui_after_score)

        except Exception as e:
            print(f"Error reading score file: {e}")

    def play_celebratory_sound(self):
        try:
            # Path to the celebratory sound effect
            sound_path = "Soundeffects/celebration.mp3"
            # Load the sound file
            data, fs = sf.read(sound_path, dtype='float32')
            # Play the sound
            sd.play(data, fs)
        except Exception as e:
            print(f"Error playing celebratory sound: {e}")

    def restore_youtube_view(self):
        # Remove the score label and show the YouTube view
        if hasattr(self, 'score_label'):
            self.layout.removeWidget(self.score_label)
            self.score_label.deleteLater()
            self.browser.show()
        print("YouTube view restored.")

    def restore_ui_after_score(self):
        # Remove the score label, show the YouTube view, Start/Stop button, and banner image
        if hasattr(self, 'score_label'):
            self.layout.removeWidget(self.score_label)
            self.score_label.deleteLater()
            self.browser.show()
            self.record_button.show()
            self.image_label.show()

            # Remove the vertical spacers
            # for i in reversed(range(self.layout.count())):
            #     item = self.layout.itemAt(i)
            #     if isinstance(item, QWidget):
            #         continue
            #     self.layout.removeItem(item)

        print("UI restored to normal.")


#####################################################################SCORE UI##################################################################  


class ScoreWindow(QWidget):
    def __init__(self, score):
        super().__init__()
        self.setWindowTitle("Karaoke Score")
        self.setWindowFlags(Qt.Window | Qt.FramelessWindowHint)
        self.setGeometry(0, 0, 1920, 1080)  # Full-screen resolution (adjust if needed)
        self.setStyleSheet("background-color: black;")

        # Create a label for the score
        self.score_label = QLabel(self)
        self.score_label.setText(f"Your Score: {score}")
        self.score_label.setAlignment(Qt.AlignCenter)
        self.score_label.setStyleSheet("color: gold; font-size: 72px; font-weight: bold;")

        layout = QVBoxLayout()
        layout.addWidget(self.score_label)
        self.setLayout(layout)

        # Play celebratory sound effect
        self.play_celebratory_sound()

        # Close the window after 5 seconds
        QTimer.singleShot(5000, self.close)

    def play_celebratory_sound(self):
        try:
            # Path to the celebratory sound effect
            sound_path = "Soundeffects/celebration.wav"
            # Load the sound file
            data, fs = sf.read(sound_path, dtype='float32')
            # Play the sound
            sd.play(data, fs)
        except Exception as e:
            print(f"Error playing celebratory sound: {e}")



# Main function
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())
